# run之前記得檢查import的檔案及日期

In [44]:
import pandas as pd
import time
import requests
from tqdm import tqdm_notebook
import json
import statistics as stat
import class_yahoof_data
import class_crsp_data

def get_nas_data(start, end):
    url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v2/get-historical-data"

    querystring = {"frequency": "1d", "filter": "history", "period1": start, "period2": end, "symbol": "%5EIXIC"}

    headers = {
        'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com",
        'x-rapidapi-key': "8099cb7d77msh3ebb39dd41af894p154acajsnabc8c78e3d42"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)

    response = json.loads(response.text)

    for i in response['prices']:
        try:
            i['volume']
        except:
            response['prices'].remove(i)

    return response['prices']

def ticker_set(ticker):
    if '.' in ticker:
        return ticker.replace('.', '-')
    elif '/' in ticker:
        return ticker.replace('/', '-')
    else:
        return ticker

if __name__ == '__main__':
    russo = pd.read_csv('C:/Users/wells/ctbc_project/Russo/09-30-2019.csv', encoding='utf8', low_memory=False)
    russo = russo[russo['Ticker'].notnull()].reset_index(drop=True)
    russo['Ticker'] = russo['Ticker'].astype('str')
    russo['Ticker'] = russo['Ticker'].apply(ticker_set)
    crsp_data = pd.read_csv('StockPrice04.csv', encoding='utf8', low_memory=False)
    start = 20190701
    end = 20190930
    
    a = str(start)+' 21:30:00'
    timeArray = time.strptime(a, "%Y%m%d %H:%M:%S")
    start_stamp = str(int(time.mktime(timeArray)))

    b = str(end)+' 21:30:00'
    timeArray = time.strptime(b, "%Y%m%d %H:%M:%S")
    end_stamp = str(int(time.mktime(timeArray)))

    nas = pd.DataFrame(get_nas_data(start_stamp, end_stamp))
    
    crsp = class_crsp_data.Get_Crsp_Data(crsp_data, nas, start, end)
    yahoo = class_yahoof_data.Get_Yahoo_Data(nas, start_stamp, end_stamp)
    
    error_ticker = []
    results = pd.DataFrame()
    for ticker in russo['Ticker']:
        data = crsp.launch(ticker)
        if data is False:
            data = yahoo.launch(ticker)
            if data is False:
                error_ticker.append(ticker)
            else:
                results = pd.concat([results, data], ignore_index=True)
        else:
            results = pd.concat([results, data], ignore_index=True)

In [67]:
import numpy as np
crsp_data[crsp_data['TICKER']=='BRK'].loc[crsp_data['date']==20140930]
crsp_data[crsp_data['DIVAMT'] != np.nan]

,PERMNO,date,TICKER,COMNAM,SHRCLS,DIVAMT,PRC,VOL
0,10001,20100104,EGAS,ENERGY INC,NaN,NaN,10.25000,18500.0
1,10001,20100105,EGAS,ENERGY INC,NaN,NaN,10.19000,23200.0
2,10001,20100106,EGAS,ENERGY INC,NaN,NaN,10.31000,18700.0
3,10001,20100107,EGAS,ENERGY INC,NaN,NaN,9.96000,29200.0
4,10001,20100108,EGAS,ENERGY INC,NaN,NaN,10.34000,25100.0
5,10001,20100111,EGAS,ENERGY INC,NaN,NaN,10.50000,23100.0
6,10001,20100112,EGAS,ENERGY INC,NaN,NaN,10.45000,5600.0
7,10001,20100113,EGAS,ENERGY INC,NaN,0.045,10.37000,5200.0
8,10001,20100114,EGAS,ENERGY INC,NaN,NaN,10.35000,11500.0
9,10001,20100115,EGAS,ENERGY INC,NaN,NaN,10.27000,13800.0


In [2]:
import pandas as pd

In [3]:
russo_r = pd.read_csv('russo_r.csv')

In [4]:
russo_r

,PERMNO,date,TICKER,COMNAM,SHRCLS,DIVAMT,PRC,VOL,shares
0,17778,20050131,BRK,BERKSHIRE HATHAWAY INC DEL,A,NaN,89900.00000,70.0,2855
1,17778,20050228,BRK,BERKSHIRE HATHAWAY INC DEL,A,NaN,90200.00000,48.0,2855
2,17778,20050331,BRK,BERKSHIRE HATHAWAY INC DEL,A,NaN,87000.00000,92.0,2855
3,13901,20050131,MO,ALTRIA GROUP INC,NaN,NaN,63.83000,1397946.0,3530211
4,13901,20050228,MO,ALTRIA GROUP INC,NaN,NaN,65.65000,1418244.0,3530211
5,13901,20050331,MO,ALTRIA GROUP INC,NaN,0.730,65.39000,1304971.0,3530211
6,89565,20050131,CMCSK,COMCAST CORP NEW,NaN,NaN,31.61000,1214243.0,4459182
7,89565,20050228,CMCSK,COMCAST CORP NEW,NaN,NaN,31.86000,959055.0,4459182
8,89565,20050331,CMCSK,COMCAST CORP NEW,NaN,NaN,33.44000,1537498.0,4459182
9,53225,20050131,WPO,WASHINGTON POST CO,B,1.850,914.50000,4537.0,150712


In [45]:
import pandas as pd
import time
import requests
from tqdm import tqdm_notebook
import json
import statistics as stat
import numpy as np

In [58]:
beta = pd.read_csv('beta.csv')
beta = beta[['DATE', 'RET', 'b_mkt', 'alpha', 'R2', 'TICKER']].drop_duplicates(keep='first', inplace=False)
beta.columns = ['date', 'RET', 'b_mkt', 'alpha', 'R2', 'ticker']
asset = pd.read_csv('asset and book value.csv')
asset = asset[['fyear', 'indfmt', 'tic', 'at', 'bkvlps', 'costat']].drop_duplicates(keep='first', inplace=False)
asset.columns = ['date', 'indfmt', 'ticker', 'at', 'bkvlps', 'costat']
asset = asset[asset['indfmt']!='FS'].loc[asset['ticker'].notnull()].reset_index(drop=True)
stock = pd.read_csv('StockPriceMonthly.csv')
stock = stock[['date', 'TICKER', 'PRC', 'SHRCLS']].drop_duplicates(keep='first', inplace=False)
stock = stock[stock['PRC'].notnull()].reset_index(drop=True)
stock.columns = ['date', 'ticker', 'price', 'class']

In [59]:
def abs_price(df_col):
    return abs(df_col)
stock['price'] = stock['price'].apply(abs_price)

In [60]:
def date_col(df_col):
    df_col = str(df_col)
    return df_col[0:6]

In [61]:
beta['date'] = beta['date'].apply(date_col)

In [62]:
asset['date'] = (asset['date']*100+12).astype('str')

In [63]:
stock['date'] = stock['date'].apply(date_col)

In [64]:
bp = asset.merge(stock, on=['date', 'ticker'], how='left')

In [65]:
bp['book/price'] = bp['bkvlps'] / bp['price']

In [10]:
bp = bp[bp['book/price'].notnull()].reset_index(drop=True)

In [66]:
bp['class'] = bp['class'].replace(np.nan, 0)

In [67]:
last_date = None
last_ticker = None
last_class = None
error_index = []
for index in tqdm_notebook(bp.index):
    date = bp.loc[index]['date']
    ticker = bp.loc[index]['ticker']
    class_ = bp.loc[index]['class']
    if last_date == date and last_ticker == ticker and last_class == class_:
        error_index.append(index)
        error_index.append(index-1)
    last_date = date
    last_ticker = ticker
    last_class = class_

In [68]:
for index in error_index:
    bp = bp.drop(index)

In [54]:
bp.to_csv('bp.csv', index=False)

In [19]:
investment = asset

In [20]:
investment = investment[investment['at'].notnull()]

In [24]:
investment.loc[:, 'last_at'] = investment.groupby(['ticker'])['at'].shift()
investment.loc[:, '2last_at'] = investment.groupby(['ticker'])['last_at'].shift()
investment.loc[:, 'last-2last'] = investment['last_at'] - investment['2last_at']

In [25]:
investment.loc[:, 'investment'] = investment['last-2last'] / investment['2last_at']

In [27]:
investment = investment[investment['investment'].notnull()].reset_index(drop=True)

In [56]:
investment.to_csv('investment.csv', index=False)

In [69]:
bp.loc[:, 'last_at'] = bp.groupby(['ticker'])['at'].shift()
bp.loc[:, '2last_at'] = bp.groupby(['ticker'])['last_at'].shift()
bp.loc[:, 'last-2last'] = bp['last_at'] - bp['2last_at']
bp.loc[:, 'investment'] = bp['last-2last'] / bp['2last_at']
bp = bp[bp['book/price'].notnull()].reset_index(drop=True)
bp = bp[bp['investment'].notnull()].reset_index(drop=True)

In [71]:
bp.to_csv('bpInvestment.csv', index=False)

In [78]:
bp = bp.merge(beta[['ticker', 'date', 'b_mkt']], on=['ticker', 'date'])

In [81]:
bp[bp['b_mkt'].notnull()].to_csv('bpInvestmentBeta.csv', index=False)

In [190]:
[1]*3

[1, 1, 1]

In [191]:
profolio = bp[['date', 'ticker', 'bkvlps', 'price', 'book/price', 'investment', 'b_mkt']]
all_profolio = pd.DataFrame()
for date, df in tqdm_notebook(profolio.groupby(['date'])):
    length = len(df)
    len30 = int(length*0.3)
    df = df.sort_values('book/price').reset_index(drop=True)
    hml_list = [1] * len30 + [0] * (length-len30)
    df.loc[:, 'hml'] = hml_list
    df = df.sort_values('investment').reset_index(drop=True)
    cma_list = [1] * len30 + [0] * (length-len30)
    df.loc[:, 'cma'] = cma_list
    all_profolio = pd.concat([all_profolio, df], ignore_index=True, sort=False)

In [192]:
all_profolio

,date,ticker,bkvlps,price,book/price,investment,b_mkt,hml,cma
0,200412,MSGI,1.4464,15.24000,0.094908,-0.848689,3.2812,1,1
1,200412,EELN,1.3377,3.38000,0.395769,-0.693726,2.0288,0,1
2,200412,LPTH,2.4787,4.41000,0.562063,-0.662006,4.3221,0,1
3,200412,INTZ,-0.0218,2.90600,-0.007502,-0.659956,3.8283,1,1
4,200412,DIGL,-0.8526,1.32000,-0.645909,-0.577244,5.0649,1,1
5,200412,ACME,4.6832,7.01000,0.668074,-0.562872,1.0088,0,1
6,200412,IOM,1.9925,5.54000,0.359657,-0.533890,0.3287,0,1
7,200412,ICOC,1.3767,3.08000,0.446981,-0.523236,0.6239,0,1
8,200412,AMRN,0.4460,2.71000,0.164576,-0.513648,0.6938,1,1
9,200412,NTIQ,7.9959,12.21000,0.654865,-0.513271,2.5533,0,1


In [197]:
return_factor = pd.read_csv(r'return_factor.csv')
return_factor = return_factor[['DATE', 'MKT']]
return_factor.columns = ['date', 'mkt']
return_factor['date'] = return_factor['date'].astype('str')

In [198]:
all_profolio.merge(return_factor, on='date')

,date,ticker,bkvlps,price,book/price,investment,b_mkt,hml,cma,mkt
0,200512,VALU,6.2047,35.32000,0.175671,-0.629614,0.1539,1,1,-0.25
1,200512,GLOV,2.8381,3.56000,0.797219,-0.615521,1.1998,0,1,-0.25
2,200512,XOMA,-0.5824,1.60000,-0.364000,-0.610770,1.9571,1,1,-0.25
3,200512,CYTR,0.1229,1.03000,0.119320,-0.590312,1.9287,1,1,-0.25
4,200512,NL,4.5237,14.09000,0.321057,-0.566999,0.8829,0,1,-0.25
5,200512,INSM,0.1583,1.97000,0.080355,-0.563565,3.3432,1,1,-0.25
6,200512,CTTY,4.0309,5.97000,0.675193,-0.556622,1.0520,0,1,-0.25
7,200512,MBAY,0.1239,1.35000,0.091778,-0.550701,1.9932,1,1,-0.25
8,200512,MFI,3.9877,3.94000,1.012107,-0.544350,0.9303,0,1,-0.25
9,200512,PANC,1.7150,6.93000,0.247475,-0.540598,0.8120,1,1,-0.25
